# Technical Analysis Helper - Usage Example

This notebook demonstrates how to use the Technical Analysis Helper for cryptocurrency price movement prediction.

In [ ]:
import sys
import os
from pathlib import Path

# Add src to Python path
sys.path.insert(0, str(Path.cwd() / 'src'))

# Import required modules
from src.models.predictor import predictor
from src.config.settings import config

## System Configuration

In [ ]:
print("=== System Configuration ===")
print(f"Feature Window Size: {config.FEATURE_WINDOW_SIZE}")
print(f"Time Windows: {config.TIME_WINDOWS}")
print(f"Classification Thresholds: {len(config.CLASSIFICATION_THRESHOLDS)} classes")
print()

print("Classification Classes:")
for class_label, (min_val, max_val) in config.CLASSIFICATION_THRESHOLDS.items():
    print(f"  Class {class_label}: {min_val}% to {max_val}%")

## Training the Model

> ⚠️ **Warning**: Training requires significant computational resources and time. 
> For demonstration purposes, use smaller parameters.

In [ ]:
# Train model with reduced parameters for demo
print("Training model...")

try:
    results = predictor.train_and_save_model(
        max_records=10000,    # Reduced from default 50000
        stride=20,            # Increased stride for fewer samples
        prediction_horizon=24
    )
    
    print("\n=== Training Results ===")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"Cross-validation accuracy: {results['cv_mean_accuracy']:.4f} ± {results['cv_std_accuracy']:.4f}")
    
    print("\nClass Confidence:")
    for class_label, confidence in results['class_confidence'].items():
        print(f"  Class {class_label}: {confidence:.4f}")
        
except Exception as e:
    print(f"Training failed: {e}")
    print("This is expected if you don't have sufficient data or resources.")

## Loading a Pre-trained Model

If you have a pre-trained model, you can load it directly:

In [ ]:
# Load existing model
print("Loading model...")

model_loaded = predictor.load_model()
if model_loaded:
    print("✓ Model loaded successfully")
else:
    print("✗ Failed to load model")
    print("Train a model first or check if model files exist")

## Making Predictions

Once the model is loaded, you can make predictions on current market data:

In [ ]:
if predictor.is_model_loaded:
    print("Making prediction on current market data...")
    
    try:
        prediction = predictor.predict_current_movement()
        
        if prediction:
            print("\n=== PRICE MOVEMENT PREDICTION ===")
            print(f"Timestamp: {prediction['timestamp']}")
            print(f"Current Price: ${prediction['current_price']:.2f}")
            print(f"Predicted Movement: {prediction['prediction_description']}")
            print(f"Confidence: {prediction['confidence']:.2%}")
            print(f"Expected Range: {prediction['price_range_min']}% to {prediction['price_range_max']}%")
            
            print("\nAll Probabilities:")
            for class_label, prob in prediction['all_probabilities'].items():
                bar = "█" * int(prob * 20)
                print(f"  Class {class_label:2s}: {prob:6.2%} |{bar:<20}|")
        else:
            print("Prediction failed")
            
    except Exception as e:
        print(f"Prediction error: {e}")
else:
    print("Model not loaded. Cannot make predictions.")

## Manual Data Exploration

You can also explore the underlying data and features:

In [ ]:
# Fetch recent data manually
from src.data.okex_fetcher import okex_fetcher
from src.data.mongodb_handler import mongo_handler

print("Fetching recent market data...")

# Get recent candlestick data
raw_data = okex_fetcher.fetch_candlesticks(bar="1H")
processed_data = okex_fetcher._process_candlestick_data(raw_data)

print(f"Fetched {len(processed_data)} recent candlesticks")

if processed_data:
    latest = processed_data[0]
    print(f"\nLatest candlestick:")
    print(f"  Timestamp: {latest['timestamp']}")
    print(f"  Open: ${latest['open']:.2f}")
    print(f"  High: ${latest['high']:.2f}")
    print(f"  Low: ${latest['low']:.2f}")
    print(f"  Close: ${latest['close']:.2f}")
    print(f"  Volume: {latest['volume']:.2f}")

## Feature Engineering Example

See how features are calculated from raw data:

In [ ]:
import pandas as pd
from src.utils.feature_engineering import feature_engineer
from src.utils.technical_indicators import tech_calculator

if len(processed_data) >= config.FEATURE_WINDOW_SIZE:
    print(f"\nCalculating features for {config.FEATURE_WINDOW_SIZE} periods...")
    
    # Take required window of data
    window_data = processed_data[-config.FEATURE_WINDOW_SIZE:]
    df = pd.DataFrame(window_data)
    
    # Calculate technical indicators
    indicators = tech_calculator.calculate_indicators(df)
    
    print(f"Calculated {len(indicators)} technical indicators")
    print("\nSample indicators:")
    
    # Show some key indicators
    key_indicators = [
        'short_rsi', 'medium_rsi', 'long_rsi',
        'short_macd', 'medium_macd', 'long_macd',
        'short_bb_position', 'medium_bb_position', 'long_bb_position'
    ]
    
    for indicator in key_indicators:
        if indicator in indicators:
            print(f"  {indicator}: {indicators[indicator]:.4f}")
else:
    print(f"Need {config.FEATURE_WINDOW_SIZE} data points, only have {len(processed_data)}")

## Summary

This notebook demonstrated:
1. System configuration and setup
2. Model training (with reduced parameters for demo)
3. Loading pre-trained models
4. Making price movement predictions
5. Manual data exploration
6. Feature engineering examples

For production use, ensure you:
- Have sufficient historical data
- Use appropriate training parameters
- Validate model performance
- Monitor predictions regularly